# Transaction Cost analysis

for different kernels

# Libraries and Data

In [92]:
import numpy as np
import pandas as pd
import sys
import os
import pickle

from project_lib.backtest import *

HOME_DIRECTORY = 'C:/Users/Harol/OneDrive/Documents/master computational finance/thesis/thesis_UCL/Code/Transaction Costs'
sys.path.append(HOME_DIRECTORY)

In [93]:
# import returns
with open(HOME_DIRECTORY + '/data/processed_daily_data/ret_subset.pkl', 'rb') as f:
    ret = pickle.load(f)

In [94]:
universe_size = 100
ret = ret.iloc[:1000, :universe_size]  # subset the data
ret = ret.iloc[(4):] # burn


In [26]:
# import weights and correlations
gauss_w = pd.read_csv("gaussian_weights.csv")
tri_w = pd.read_csv("triangular_weights.csv")
epan_w = pd.read_csv("epanechnikov_weights.csv")

gauss_c = pd.read_csv("gaussian_correlations.csv")
tri_c = pd.read_csv("triangular_correlations.csv")
epan_c = pd.read_csv("epanechnikov_correlations.csv")

dfs = [gauss_w, tri_w, epan_w, gauss_c, tri_c, epan_c]

In [27]:
for df in dfs:
    df.set_index("date", inplace=True)

In [49]:
kernels = ["gaussian","triangular","epanechnikov"]

## preprocessing

In [28]:
def cleaning(x):
    a = [ele for ele in x.strip("[]").split(" ") if ele.strip()]
    a = [elem.replace("\n","") for elem in a]
    return np.asarray(a,dtype=float)

In [29]:
gauss_w  = gauss_w.applymap(cleaning)
tri_w = tri_w.applymap(cleaning)
epan_w = epan_w.applymap(cleaning)

## correcting a mistake regarding the correlations

In [ ]:
K_pl = np.linalg.inv(cov_R_half) @ (results.T @ results) @ np.linalg.inv(cov_R_half)
        
        # make sure it's sorted
        eigen_val, eigen_vec = np.linalg.eig(K_pl)
        order = np.argsort(eigen_val)[::-1]
        idx = np.empty_like(order)
        idx[order] = np.arange(len(order))
        eigen_vec[:] = eigen_vec[:, idx] 

In [101]:
# step 1: all column elements as one matrix
# step 2: eigenvalues
# step 3: eigenvalues in place
#for kernel in kernels:
for kernel in kernels:
    print(kernel)
    cp = all_weights[kernel].copy()
    for row in range(cp.shape[0]):
        cov_R_half = get_cov(ret.iloc[row:(row+250),:], method="sample", square_root=True)
        # re-extract the weights
        w = cov_R_half.T @ np.array(gauss_w.iloc[0,:].to_list()).T
        
        K = w @ w.T
        
        eigenval, eigenvec = np.linalg.eig(K)
        
        for col in range(all_corr[kernel].shape[1]):
            all_corr[kernel].iloc[row,col] = eigenval[col]


gaussian
triangular
epanechnikov


# No Transaction Costs Performance

In [34]:
all_weights = {'gaussian':gauss_w, 'triangular':tri_w,'epanechnikov':epan_w}
all_corr = {'gaussian':gauss_c, 'triangular':tri_c,'epanechnikov':epan_c}

In [206]:
def update_weights(new_weights,old_weights, corr, tcost, risk_aversion, pf_variance):
    """
        rebalance the weights depending on rebalancing costs, risk aversion, and portfolio variance
        
        inputs:
                new_weights   : matrix of all weights calculated using CCA at t [m x m]
                old_weights   : total portfolio weights at t-1                  [1 x m]
                corr          : correlations corresponding to new_weights at t  [1 x m]
                tcost         : transaction cost parameter                      [1 x 1]
                risk_aversion : risk aversion parameter                         [1 x 1]
                pf_variance   : current portfolio variance at t-1               [1 x 1]
        outputs:
                change_w : change in weights (compared to old weights) [1 x m]
    """
    # step 1 : multiply each portfolio by its correlation
    w = new_weights*np.array(corr)
    # step 2 : calculate total weights
    total_w = w.sum()
    # step 3 : calculate weights change
    change_w = old_weights - total_w
    # step 4 : initial transaction cost
    rebalance_cost = tcost * np.sum(np.abs(change_w))
    # step 5 : calculate which portfolios to include iteratively
    trade = False
    while(trade==False):
        # check whether it is worth rebalancing given total current turnover
        if np.sum(np.abs(change_w)) - (1/risk_aversion) * rebalance_cost / pf_variance < 0:
            # remove last canonical portfolio
            # unless we decide not to trade any portfolio -> end loop
            if w.shape[1]==0:
                trade = True
            else:
                w = np.delete(w, -1, 1)
                total_w = w.sum()
                change_w = old_weights - total_w
                rebalance_cost = tcost * np.sum(np.abs(change_w))
        else:
            trade = True
    # return the change in weights
    return change_w

In [ ]:
# function to iteratively calculate portfolio weights & profitability
def backtest_cca():
    return 0